# 1. Initializations

## 1.1 General imports

In [ ]:
### Global
import logging
from smartcheck.logger_config import setup_logger
setup_logger(logging.INFO)

### Data management
import pandas as pd
import numpy as np

### Machine Learning
# pipelines
from sklearn.pipeline import Pipeline

# metrics and evaluation
from scipy.stats import anderson, pearsonr, shapiro, normaltest, levene, kruskal
import statsmodels.api as sm
import statsmodels.formula.api as smf

### Data Viz
# graphical basics
import matplotlib.pyplot as plt
%matplotlib inline

# graphical seaborn
import seaborn as sns

# graphical missingno
import missingno as msno

## 1.2 General dataframe functions

In [ ]:
import smartcheck.dataframe_common as dfc
import smartcheck.preprocessing_project_specific as pps

# 2. Loading and Data Enrichment

## 2.1 Loading of refactored velo comptage data 2024/2025

In [ ]:
df_cpt_raw = dfc.load_dataset_from_config('velo_comptage_refactored_data', sep=',', index_col=0)

if df_cpt_raw is not None and isinstance(df_cpt_raw, pd.DataFrame):
    df_cpt = df_cpt_raw.copy()

## 2.2 Data refactoring and additions

In [ ]:
keep_cols = [
    "identifiant_du_compteur",
    "nom_du_site_de_comptage",
    "comptage_horaire",
    "date_et_heure_de_comptage",
    "orientation_compteur",
    "latitude",
    "longitude",
    "arrondissement",
]
import smartcheck.preprocessing_project_specific as pps
preprocessor = Pipeline([
    ("filter_columns", pps.ColumnFilterTransformer(columns_to_keep=keep_cols)),
    ("add_datetime_features", pps.DatetimePreprocessingTransformer(timestamp_col="date_et_heure_de_comptage")),
    ("add_holiday", pps.HolidayFromDatetimeTransformer(datetime_col="date_et_heure_de_comptage_local")),
    ("add_school_vacation", pps.SchoolHolidayTransformer(datetime_col="date_et_heure_de_comptage_local")),
    ("add_weather_data", pps.WeatherDataEnrichmentTransformer(
        lat_col="latitude",
        lon_col="longitude",
        datetime_col="date_et_heure_de_comptage_utc"
    )),
    ("normalize_columns", pps.ColumnNameNormalizerTransformer()),
    ("add_weather_category", pps.MeteoCodePreprocessingTransformer(code_col="weather_code_wmo_code")),
])

df_raw = preprocessor.fit_transform(df_cpt)
if df_raw is not None and isinstance(df_raw, pd.DataFrame):
    df = df_raw.copy()

#### Intermediate backup

In [ ]:
# backup
df_bckp_orig = df.copy()

In [ ]:
# Restore (if needed to recover)
df = df_bckp_orig.copy()

## 2.3 Explore and verify enriched datas

#### General checks

In [ ]:
# Infos générales
display(df.head())
dfc.log_general_info(df)
nb_first, nb_total = dfc.detect_and_log_duplicates_and_missing(df)
if nb_first != nb_total:
    print(dfc.duplicates_index_map(df))

#### Check missing value

In [ ]:
# Représentation des valeur NA graphiquement
msno.matrix(df_bckp_orig)

#### Check public holidays

In [ ]:
display(df_bckp_orig[df_bckp_orig.jour_ferie==1])

#### Check school holidays

In [ ]:
groupby_dy_vs = df_bckp_orig.groupby(['date_et_heure_de_comptage_day_of_year', 'vacances_scolaires']).identifiant_du_compteur.count().reset_index()
display(groupby_dy_vs.head(20))
dfc.display_variable_info(df.vacances_scolaires)

#### Check descriptions and correlation of variables

In [ ]:
df_desc_num = df_bckp_orig.select_dtypes(include=np.number).describe()
display(df_desc_num)
df_desc_cat = df_bckp_orig.select_dtypes(include='object').describe()
display(df_desc_cat)
df_cr = df_bckp_orig.select_dtypes(include=np.number).corr()
display(df_cr)


# 3. Analyse statistique

In [ ]:
# Anderson :
# Hypothèse nulle H0 : -> la distribution est normale
# Hypothèse alternative H1 : -> on réfute la distribution normale
# Explication du test : si pour une tolérance donnée, la statistique de test est supérieure au seuil critique alors on rejette 
# statistiquement H0 et on accepte H1 sinon on n'a PAS de preuve statistique contre H0 et on ne peut rien conclure

# ici tous les seuil même les plus large (15%) rejette l'hypothese de normalité (ce qu'on voyait déjà à l'oeil nu graphiquement)
result = anderson(df_bckp_orig['comptage_horaire'])
print(f"Statistique de test = {result.statistic:.4f}")  # type: ignore
for i in range(len(result.critical_values)):  # type: ignore
    sig_level = result.significance_level[i]  # type: ignore
    crit_value = result.critical_values[i]  # type: ignore
    if result.statistic > crit_value:  # type: ignore
        print(f"❌ À {sig_level}% : rejet de la normalité (stat > seuil critique {crit_value:.3f})")
    else:
        print(f"✅ À {sig_level}% : pas de preuve contre la normalité")

#### Pearson (quantitatives against quantitatives)

In [ ]:
# Pearson
# Hypothèse nulle H0 : -> les deux variables quantitatives ne sont pas correlées (corrélation est nulle)
# Hypothèse alternative H1 : -> il existe une correlation (corrélation <> nulle)
# Explication du test : si la statistique de test a une p-valeur inférieure à un seuil de tolérance (0,05) alors on rejette 
# statistiquement H0 et on accepte H1 (il existe une correlation) sinon on n'a PAS de preuve statistique contre H0 et on ne peut rien conclure
coeff_corr, p_valeur = pearsonr (x=df_bckp_orig['comptage_horaire'], y=df_bckp_orig['latitude'])
print(f"comptage_horaire/latitude: coefficient de correlation[{coeff_corr}] et p-valeur[{p_valeur}]")
coeff_corr, p_valeur = pearsonr (x=df_bckp_orig['comptage_horaire'], y=df_bckp_orig['longitude'])
print(f"comptage_horaire/longitude: coefficient de correlation[{coeff_corr}] et p-valeur[{p_valeur}]")

corr_matrix = df_bckp_orig.select_dtypes(include='number').corr(method='pearson')
plt.figure(figsize=(12,10))
sns.heatmap(corr_matrix, annot=True, fmt=".2f", cmap="coolwarm", mask=np.triu(corr_matrix), center=0)
plt.title("Matrice de corrélation (Pearson)")
plt.show()

#### ANOVA (quantitative against qualitatives)

In [ ]:
col_cat = [
    'arrondissement',
    'orientation_compteur',
    'nom_du_site_de_comptage',
    'weather_code_wmo_code_category',
    'vacances_scolaires'
]

In [ ]:
# ANOVA (NB : a priori le test ANOVA n'est pas robuste si notre variable quantitative ne suit pas une loi normale)
# Hypothèse nulle H0 : -> pas d'effet significatif de la variable qualitative sur la variable quantitative
# Hypothèse alternative H1 : -> il y a un effet significatif de la variable qualitative sur la variable quantitative
# Explication du test : si la statistique de test a une p-valeur inférieure a un seuil de tolérance (0,05) alors on rejette 
# statistiquement H0 et on accepte H1 (il y a un effet significatif de la variable qualitative sur la variable quantitative) 
# sinon on n'a PAS de preuve statistique contre H0 et on ne peut rien conclure

result = smf.ols(f'comptage_horaire ~ {' + '.join(col_cat)}', data=df).fit()
display(sm.stats.anova_lm(result))

# - Les F-statistiques de l’ANOVA sont élevées → chaque variable apporte significativement à l’explication de la variance de 
# comptage_horaire


In [ ]:
# Analyse des coefficients du modèle
# R² = 0.285 → le modèle explique 28,5 % de la variance du comptage horaire
# beaucoup de p-values de modalités sont très haute (modalités non correlées)
# probablement un viol des conditions de l'ANOVA (distribution normale entre les modalités)
result.summary()

In [ ]:
# Résidus
residuals = result.resid

# 1.1 Histogramme des résidus
plt.figure(figsize=(8, 5))
sns.histplot(residuals, kde=True, bins=50)
plt.title("Distribution des résidus du modèle ANOVA")
plt.xlabel("Résidus")
plt.ylabel("Fréquence")
plt.show()

# 1.2 QQ-plot
sm.qqplot(residuals, line='s')
plt.title("QQ-plot des résidus")
plt.show()

# 1.3 Test de normalité de Shapiro (si n <= 5000)
if len(residuals) <= 5000:
    stat, p = shapiro(residuals)
    print(f"Test de Shapiro-Wilk : p-value = {p:.4f}")
else:
    stat, p = normaltest(residuals)
    print(f"Test de D'Agostino : p-value = {p:.4f}")

# 1.4 Test d'homogénéité des variances (Levene)
for col in col_cat:
    stat_levene, p_levene = levene(*[group["comptage_horaire"].values
                                        for _, group in df.groupby(col)])
    print(f"Test de Levene ({col}) : p-value = {p_levene:.4f}")


#### Kruskal-Wallis (quantitative against qualitatives, non parametric)

In [ ]:
# On récupère les groupes de comptage_horaire par arrondissement
for col in col_cat:
    grouped = [group["comptage_horaire"].values for _, group in df.groupby(col)]

    # Exécute le test de Kruskal-Wallis
    stat_kw, p_kw = kruskal(*grouped)
    print(f"Test de Kruskal-Wallis : H = {stat_kw:.2f}, p-value = {p_kw:.4f}")

    plt.figure(figsize=(15, 6))
    sns.boxplot(x=col, y='comptage_horaire', data=df)
    plt.title(f"Boxplot du comptage horaire par {col}")
    plt.xticks(rotation=90)
    plt.tight_layout()
    plt.show()

## 4. Data backup on file

#### Suppression des colonnes de données périodiques qui doivent être recalculées (allègement du dataset)

In [ ]:
df = df.drop(
    columns=[
        'date_et_heure_de_comptage_utc',
        'date_et_heure_de_comptage_local',
        'date_et_heure_de_comptage_year',
        'date_et_heure_de_comptage_month',
        'date_et_heure_de_comptage_day',
        'date_et_heure_de_comptage_day_of_year',
        'date_et_heure_de_comptage_day_of_week',
        'date_et_heure_de_comptage_hour',
        'date_et_heure_de_comptage_week',
        'date_et_heure_de_comptage_dayname',
        'date_et_heure_de_comptage_monthname',
    ]
)

#### Sauvegarde du dataset en CSV

In [ ]:
df.to_csv("comptage-velo-donnees-compteurs-2024-2025_Enriched_ML-ready_data.csv")